## estrazione features da reti pretrainate

In [ ]:
import numpy as np
import collections
import nrrd
import os
import pandas as pd
from keras.preprocessing import image
from keras.models import load_model
import nrrd
import collections
from keras.preprocessing import image
from skimage.transform import resize
import cv2
from keras.models import load_model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input

## 2D VGG CUT

In [ ]:
model = VGG19(weights='imagenet', include_top=False)

# Patientes folder
pathdicom = "\\Users\\bsbar\\Desktop\\pazienti_nrrd"
pathroi = "\\Users\\bsbar\\Desktop\\Tesi\\ROI"

# Function to find coordinates
def maskcroppingbox(mask_slice):
    non_zero_coords = np.argwhere(mask_slice)
    (ystart, xstart), (ystop, xstop) = non_zero_coords.min(axis=0), non_zero_coords.max(axis=0) + 1
    return (ystart, xstart), (ystop, xstop)

# Preprocess of slices
def extract_squares_for_all_slices(image_array, mask_array, target_size=224):
    slices_with_plaque = []

    # Iterate over all slices
    for i in range(image_array.shape[0]):
        mask_slice = mask_array[i]
        image_slice = image_array[i]

        if np.sum(mask_slice) > 0:
            (ystart, xstart), (ystop, xstop) = maskcroppingbox(mask_slice)

            # Find the center of the plaque
            center_y = (ystart + ystop) // 2
            center_x = (xstart + xstop) // 2

            # Crop the image around the plaque
            half_side = min(center_y, center_x, 30) 
            roi_image = image_slice[
                max(0, center_y-half_side):min(center_y+half_side, image_slice.shape[0]), 
                max(0, center_x-half_side):min(center_x+half_side, image_slice.shape[1])
            ]
            
            # Resize to 224*224
            roi_image_resized = resize(roi_image, (target_size, target_size), order=1, anti_aliasing=True)

        else:
            roi_image_resized = image_array[i]

        slices_with_plaque.append(roi_image_resized)

    return slices_with_plaque

# Extract features
def featureextraction(image_array, mask_array):
    roi_images_resized = extract_squares_for_all_slices(image_array, mask_array)

    # Find the largest slice
    slice_sums = np.sum(mask_array, axis=(1, 2))
    largest_slice_index = np.argmax(slice_sums)
    largest_slice_image = roi_images_resized[largest_slice_index]
    #print(image_array.shape)
    x = image.img_to_array(largest_slice_image)

    # Repeat the channel
    x = np.repeat(x, 3, axis=-1)  
    # Add batch dimension
    x = np.expand_dims(x, axis=0)
    # preprocess using the model's function
    x = preprocess_input(x)

    # extract the feature maps
    base_model_pool_features = model.predict(x)
    feature_map = base_model_pool_features[0]

    feature_map = feature_map.transpose((2, 1, 0))

    ## Global Max Pooling
    features = np.max(feature_map, -1)
    features = np.max(features, -1)
    # Collect the features
    deeplearningfeatures = collections.OrderedDict()
    for ind_, f_ in enumerate(features):
        deeplearningfeatures[str(ind_)] = f_

    return deeplearningfeatures


# Main cycle
featureDict = {}
for s in os.listdir(pathdicom):
    print(s)
    filename = os.path.join(pathdicom, s)

        
    for t in os.listdir(filename):

        pathdicomnew = os.path.join(pathdicom, s, t)
        readdatadicom, header = nrrd.read(pathdicomnew, index_order='C')

    pathroinew = os.path.join(pathroi, s)
    for g in os.listdir(pathroinew):

        troi = os.path.join(pathroi, s, g)
        readdatanrrd, header2 = nrrd.read(troi, index_order='C')

    
    deeplearningfeatures = featureextraction(readdatadicom,readdatanrrd) 

    result = deeplearningfeatures
    key = list(result.keys())
    key = key[0:]
        
    feature = []
    for jind in range(len(key)):
        feature.append(result[key[jind]])
        
    featureDict[s] = feature
    dictkey = key
    print(s)


dataframe = pd.DataFrame.from_dict(featureDict, orient='index', columns=dictkey)
dataframe.to_csv('C:\\Users\\bsbar\\Desktop\\INCEPTION_SingolaSlice_Imagenet_Ritagliata.csv')
    



## 2D VGG FULL

In [ ]:
model = VGG19(weights='imagenet', include_top=False)
# Patients folder
pathdicom = "\\Users\\bsbar\\Desktop\\pazienti_nrrd"
pathroi = "\\Users\\bsbar\\Desktop\\Tesi\\ROI"

# Extract features
def featureextraction(image_array, mask_array):

    roi_images = image_array 
    mask_slices = mask_array
    
    # Lagest slice
    slice_sums = np.sum(mask_slices, axis=(1, 2)) 
    largest_slice_index = np.argmax(slice_sums)

    print(largest_slice_index)
    print(roi_images.shape)

    largest_slice_image = roi_images[largest_slice_index]

    x = image.img_to_array(largest_slice_image)
    # Resize to the correct dimension
    x = cv2.resize(x, (224, 224), interpolation=cv2.INTER_LINEAR)
    x = np.repeat(x[..., np.newaxis], 3, axis=-1)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Extract the fetaure maps
    base_model_pool_features = model.predict(x)

    feature_map = base_model_pool_features[0]
    feature_map = feature_map.transpose((2, 1, 0))
    
    ## Global Max Pooling
    features = np.max(feature_map, -1)
    features = np.max(features, -1)

    deeplearningfeatures = collections.OrderedDict()
    
    for ind_, f_ in enumerate(features):
        deeplearningfeatures[str(ind_)] = f_
         
    return deeplearningfeatures


# Main cycle
featureDict = {}
for s in os.listdir(pathdicom):
    print(s)
    filename = os.path.join(pathdicom, s)

        
    for t in os.listdir(filename):

        pathdicomnew = os.path.join(pathdicom, s, t)
        readdatadicom, header = nrrd.read(pathdicomnew, index_order='C')

    pathroinew = os.path.join(pathroi, s)
    for g in os.listdir(pathroinew):

        troi = os.path.join(pathroi, s, g)
        readdatanrrd, header2 = nrrd.read(troi, index_order='C')

    
    deeplearningfeatures = featureextraction(readdatadicom,readdatanrrd) 

    result = deeplearningfeatures
    key = list(result.keys())
    key = key[0:]
        
    feature = []
    for jind in range(len(key)):
        feature.append(result[key[jind]])
        
    featureDict[s] = feature
    dictkey = key
    print(s)
    
dataframe = pd.DataFrame.from_dict(featureDict, orient='index', columns=dictkey)
dataframe.to_csv('C:\\Users\\bsbar\\Desktop\\INCEPTION_SingolaSlice_Imagenet_fullimage.csv')

## RADIMAGENET 2D FULL

In [ ]:
#"../PreTrained_Models/RadImageNet-IRV2_notop.h5"
#"../PreTrained_Models/RadImageNet-ResNet50_notop.h5"
#"../PreTrained_Models/RadImageNet-InceptionV3_notop.h5"

# Load the pre trained model, you can choose the network by copy and pasting the paths above
model_path = "../PreTrained_Models/RadImageNet-InceptionV3_notop.h5"
model = load_model(model_path)

# Patients folder
pathdicom = "\\Users\\bsbar\\Desktop\\pazienti_nrrd"
pathroi = "\\Users\\bsbar\\Desktop\\Tesi\\ROI"

# Extract features
def featureextraction(image_array, mask_array):

    # Largest slice
    slice_sums = np.sum(mask_array, axis=(1, 2))
    largest_slice_index = np.argmax(slice_sums)
    largest_slice_image = image_array[largest_slice_index]
    x = image.img_to_array(largest_slice_image)
    min_val = np.min(x)
    max_val = np.max(x)

    # Min max normalization
    x = (x - min_val) / (max_val - min_val)

    # Resize to 224*224
    x = cv2.resize(x, (224, 224))

    # Add channel dimension
    x = np.expand_dims(x, axis=-1)

    # Repeat the channel
    x = np.repeat(x, 3, axis=-1)    
    
    # Add batch dimension
    x = np.expand_dims(x, axis=0)
    
    # Extract the features
    base_model_pool_features = model.predict(x)
    feature_map = base_model_pool_features[0]
    feature_map = feature_map.transpose((2, 1, 0))
    
    ## Global Max Pooling
    features = np.max(feature_map, -1)
    features = np.max(features, -1)
    
    deeplearningfeatures = collections.OrderedDict()
    for ind_, f_ in enumerate(features):
        deeplearningfeatures[str(ind_)] = f_

    return deeplearningfeatures

# Main cycle
featureDict = {}
for s in os.listdir(pathdicom):
    filename = os.path.join(pathdicom, s)

    for t in os.listdir(filename):
        pathdicomnew = os.path.join(pathdicom, s, t)
        readdatadicom, header = nrrd.read(pathdicomnew, index_order='C')

    pathroinew = os.path.join(pathroi, s)
    for g in os.listdir(pathroinew):
        troi = os.path.join(pathroi, s, g)
        readdatanrrd, header2 = nrrd.read(troi, index_order='C')

    deeplearningfeatures = featureextraction(readdatadicom, readdatanrrd)

    result = deeplearningfeatures
    key = list(result.keys())

    feature = []
    for jind in range(len(key)):
        feature.append(result[key[jind]])

    featureDict[s] = feature
    dictkey = key
    print(s)

#dataframe = pd.DataFrame.from_dict(featureDict, orient='index', columns=dictkey)
#dataframe.to_csv('C:\\Users\\bsbar\\Desktop\\INCEPTION_SliceMaggiore_NuoviPesi_fullimage_GAP.csv')


## RADIMAGENET 2D CUT

In [ ]:
#"../PreTrained_Models/RadImageNet-IRV2_notop.h5"
#"../PreTrained_Models/RadImageNet-ResNet50_notop.h5"
#"../PreTrained_Models/RadImageNet-InceptionV3_notop.h5"

# Load the pre trained model, you can choose the network by copy and pasting the paths above
model_path = "../PreTrained_Models/RadImageNet-InceptionV3_notop.h5"
model = load_model(model_path)

# Patients folder
pathdicom = "\\Users\\bsbar\\Desktop\\pazienti_nrrd"
pathroi = "\\Users\\bsbar\\Desktop\\Tesi\\ROI"

# Find coordinates
def maskcroppingbox(mask_slice):
    non_zero_coords = np.argwhere(mask_slice)
    (ystart, xstart), (ystop, xstop) = non_zero_coords.min(axis=0), non_zero_coords.max(axis=0) + 1
    return (ystart, xstart), (ystop, xstop)

# Preprocess the slices
def extract_squares_for_all_slices(image_array, mask_array, target_size=224):
    slices_with_plaque = []
    for i in range(image_array.shape[0]):
        mask_slice = mask_array[i]
        image_slice = image_array[i]

        if np.sum(mask_slice) > 0:
            (ystart, xstart), (ystop, xstop) = maskcroppingbox(mask_slice)
            center_y = (ystart + ystop) // 2
            center_x = (xstart + xstop) // 2

            half_side = min(center_y, center_x, 30)

            # min max normalization
            image_slice_normalized = (image_slice - np.min(image_slice)) / (np.max(image_slice) - np.min(image_slice)) 
            roi_image = image_slice_normalized[
                max(0, center_y-half_side):min(center_y+half_side, image_slice.shape[0]), 
                max(0, center_x-half_side):min(center_x+half_side, image_slice.shape[1])
            ]
            
            # Resize
            roi_image_resized = resize(roi_image, (target_size, target_size), order=1, anti_aliasing=True)

        else:
            roi_image_resized = image_array[i]

        slices_with_plaque.append(roi_image_resized)

    return slices_with_plaque

# Extract features
def featureextraction(image_array, mask_array):
    roi_images_resized = extract_squares_for_all_slices(image_array, mask_array)

    # Find the largest slice
    slice_sums = np.sum(mask_array, axis=(1, 2))
    largest_slice_index = np.argmax(slice_sums)
    largest_slice_image = roi_images_resized[largest_slice_index]
    x = image.img_to_array(largest_slice_image)

    # Repeat the channel
    x = np.repeat(x, 3, axis=-1)    
    
    # Add batch dimension
    x = np.expand_dims(x, axis=0)

    # Estrazione delle feature principali
    base_model_pool_features = model.predict(x)
    feature_map = base_model_pool_features[0]

    feature_map = feature_map.transpose((2, 1, 0))

    ## Global Max Pooling
    features = np.max(feature_map, -1)
    features = np.max(features, -1)

    deeplearningfeatures = collections.OrderedDict()
    for ind_, f_ in enumerate(features):
        deeplearningfeatures[str(ind_)] = f_

    return deeplearningfeatures


# Main cycle
featureDict = {}
for s in os.listdir(pathdicom):
    filename = os.path.join(pathdicom, s)

    for t in os.listdir(filename):
        pathdicomnew = os.path.join(pathdicom, s, t)
        readdatadicom, header = nrrd.read(pathdicomnew, index_order='C')

    pathroinew = os.path.join(pathroi, s)
    for g in os.listdir(pathroinew):
        troi = os.path.join(pathroi, s, g)
        readdatanrrd, header2 = nrrd.read(troi, index_order='C')

    deeplearningfeatures = featureextraction(readdatadicom, readdatanrrd)

    result = deeplearningfeatures
    key = list(result.keys())

    feature = []
    for jind in range(len(key)):
        feature.append(result[key[jind]])

    featureDict[s] = feature
    dictkey = key
    print(s)

#dataframe = pd.DataFrame.from_dict(featureDict, orient='index', columns=dictkey)
#dataframe.to_csv('C:\\Users\\bsbar\\Desktop\\RESNET_SliceMaggiore_NuoviPesi_Ritagliata.csv')


# MULTIPLE SLICE


### 2.5D VGG CUT

In [ ]:
model = VGG19(weights='imagenet', include_top=False)

pathdicom = "\\Users\\bsbar\\Desktop\\pazienti_nrrd"
pathroi = "\\Users\\bsbar\\Desktop\\Tesi\\ROI"


def maskcroppingbox(mask_slice):
    non_zero_coords = np.argwhere(mask_slice)
    (ystart, xstart), (ystop, xstop) = non_zero_coords.min(axis=0), non_zero_coords.max(axis=0) + 1
    return (ystart, xstart), (ystop, xstop)

def featureextraction(image_array, mask_array, patient_id):
    # List to contain all the slices
    batch_slices = []
    original_slice_indices = []

    # Iterate over every slice
    for slice_idx in range(image_array.shape[0]): 
        image_slice = image_array[slice_idx]
        mask_slice = mask_array[slice_idx]

        # If the mask is not empty preprocess the slice
        if np.any(mask_slice):
            
            (ystart, xstart), (ystop, xstop) = maskcroppingbox(mask_slice)
            center_y = (ystart + ystop) // 2
            center_x = (xstart + xstop) // 2
            half_side = min(center_y, center_x, 30)
            roi_image = image_slice[
                max(0, center_y-half_side):min(center_y+half_side, image_slice.shape[0]), 
                max(0, center_x-half_side):min(center_x+half_side, image_slice.shape[1])
            ]
            roi_image = roi_image.astype(np.float32)
            roi_image_resized = resize(roi_image, (224, 224), order=1, anti_aliasing=True)

            x = image.img_to_array(roi_image_resized)
            x = np.repeat(x, 3, axis=-1) 

            x = preprocess_input(x)
            batch_slices.append(x)
            # Save the index for the csv
            original_slice_indices.append(slice_idx)

    batch_slices = np.array(batch_slices)

    # The whole batch is passed to the pre trained network
    base_model_pool_features = model.predict(batch_slices)

    all_features = []

    # Consider only the slices based on the original index
    for i in range(base_model_pool_features.shape[0]):
        feature_map = base_model_pool_features[i]
        feature_map = feature_map.transpose((2, 1, 0))
        
        ## Global Max Pooling
        features = np.max(feature_map, -1)
        features = np.max(features, -1)

        feature_entry = {'Patient': patient_id, 'Slice': original_slice_indices[i]}
        for ind_, f_ in enumerate(features):
            feature_entry[f'Feature_{ind_}'] = f_

        all_features.append(feature_entry)

    return all_features


all_feature_dicts = []

for s in os.listdir(pathdicom):
    print(f"Processing patient: {s}")
    filename = os.path.join(pathdicom, s)

    for t in os.listdir(filename):
        pathdicomnew = os.path.join(pathdicom, s, t)
        readdatadicom, header = nrrd.read(pathdicomnew, index_order='C')

    pathroinew = os.path.join(pathroi, s)
    for g in os.listdir(pathroinew):
        troi = os.path.join(pathroi, s, g)
        readdatanrrd, header2 = nrrd.read(troi, index_order='C')
    patient_features = featureextraction(readdatadicom, readdatanrrd, patient_id=s)
    all_feature_dicts.extend(patient_features)

# Create and save the final csv
#dataframe = pd.DataFrame(all_feature_dicts)
#dataframe.to_csv('C:\\Users\\bsbar\\Desktop\\RESNET_ALL_SLICES_Imagenet_ritagliata.csv', index=False)
    



### 2.5D VGG FULL

In [ ]:
model = VGG19(weights='imagenet', include_top=False)

pathdicom = "\\Users\\bsbar\\Desktop\\pazienti_nrrd"
pathroi = "\\Users\\bsbar\\Desktop\\Tesi\\ROI"

def featureextraction(image_array, mask_array, patient_id):
    batch_slices = []
    original_slice_indices = []

    for slice_idx in range(image_array.shape[0]):
        image_slice = image_array[slice_idx]
        mask_slice = mask_array[slice_idx]

        if np.any(mask_slice):
            
            # Resize and preprocessing
            image_slice = image_slice.astype(np.float32)
            roi_image_resized = cv2.resize(image_slice, (224, 224), interpolation=cv2.INTER_LINEAR)
            x = image.img_to_array(roi_image_resized)
            x = np.repeat(x, 3, axis=-1) 
            x = preprocess_input(x)
            batch_slices.append(x)
            original_slice_indices.append(slice_idx)

    batch_slices = np.array(batch_slices)
    # Extract the feature maps
    base_model_pool_features = model.predict(batch_slices)

    all_features = []
    for i in range(base_model_pool_features.shape[0]):
        feature_map = base_model_pool_features[i]
        feature_map = feature_map.transpose((2, 1, 0))
        
        ## Global Max Pooling
        features = np.max(feature_map, -1)
        features = np.max(features, -1)

        feature_entry = {'Patient': patient_id, 'Slice': original_slice_indices[i]}
        for ind_, f_ in enumerate(features):
            feature_entry[f'Feature_{ind_}'] = f_

        all_features.append(feature_entry)

    return all_features


all_feature_dicts = []

for s in os.listdir(pathdicom):
    print(f"Processing patient: {s}")
    filename = os.path.join(pathdicom, s)

    for t in os.listdir(filename):
        pathdicomnew = os.path.join(pathdicom, s, t)
        readdatadicom, header = nrrd.read(pathdicomnew, index_order='C')

    pathroinew = os.path.join(pathroi, s)
    for g in os.listdir(pathroinew):
        troi = os.path.join(pathroi, s, g)
        readdatanrrd, header2 = nrrd.read(troi, index_order='C')

    patient_features = featureextraction(readdatadicom, readdatanrrd, patient_id=s)
    all_feature_dicts.extend(patient_features)

# create and save the final csv
#dataframe = pd.DataFrame(all_feature_dicts)
#dataframe.to_csv('C:\\Users\\bsbar\\Desktop\\INCEPTION_ALL_SLICES_Imagenet_fullimage.csv', index=False)

### RADIMAGENET 2.5D CUT

In [ ]:
#"../PreTrained_Models/RadImageNet-IRV2_notop.h5"
#"../PreTrained_Models/RadImageNet-ResNet50_notop.h5"
#"../PreTrained_Models/RadImageNet-InceptionV3_notop.h5"

# Load the pre trained model, you can choose the network by copy and pasting the paths above
model_path = "../PreTrained_Models/RadImageNet-InceptionV3_notop.h5"
model = load_model(model_path)

# Patients folder
pathdicom = "\\Users\\bsbar\\Desktop\\pazienti_nrrd"
pathroi = "\\Users\\bsbar\\Desktop\\Tesi\\ROI"

def maskcroppingbox(mask_slice):
    non_zero_coords = np.argwhere(mask_slice)
    (ystart, xstart), (ystop, xstop) = non_zero_coords.min(axis=0), non_zero_coords.max(axis=0) + 1
    return (ystart, xstart), (ystop, xstop)

def featureextraction(image_array, mask_array, patient_id):
    batch_slices = []
    original_slice_indices = []

    for slice_idx in range(image_array.shape[0]):
        image_slice = image_array[slice_idx]
        mask_slice = mask_array[slice_idx]

        if np.any(mask_slice):
            (ystart, xstart), (ystop, xstop) = maskcroppingbox(mask_slice)
            center_y = (ystart + ystop) // 2
            center_x = (xstart + xstop) // 2

            half_side = min(center_y, center_x, 30)

            # min max norm and create the final slice
            image_slice_normalized = (image_slice - np.min(image_slice)) / (np.max(image_slice) - np.min(image_slice)) 
            roi_image = image_slice_normalized[
                max(0, center_y-half_side):min(center_y+half_side, image_slice.shape[0]), 
                max(0, center_x-half_side):min(center_x+half_side, image_slice.shape[1])
            ]
            
            roi_image_resized = resize(roi_image, (224, 224), order=1, anti_aliasing=True)
            x = image.img_to_array(roi_image_resized)
            x = np.repeat(x, 3, axis=-1)          
            batch_slices.append(x)
            original_slice_indices.append(slice_idx)

    batch_slices = np.array(batch_slices)

    base_model_pool_features = model.predict(batch_slices)

    all_features = []
    for i in range(base_model_pool_features.shape[0]):
        feature_map = base_model_pool_features[i]
        feature_map = feature_map.transpose((2, 1, 0))
        
        ## Global Max Pooling
        features = np.max(feature_map, -1)
        features = np.max(features, -1)

        feature_entry = {'Patient': patient_id, 'Slice': original_slice_indices[i]}
        for ind_, f_ in enumerate(features):
            feature_entry[f'Feature_{ind_}'] = f_

        all_features.append(feature_entry)

    return all_features


all_feature_dicts = []

for s in os.listdir(pathdicom):
    print(f"Processing patient: {s}")
    filename = os.path.join(pathdicom, s)

    for t in os.listdir(filename):
        pathdicomnew = os.path.join(pathdicom, s, t)
        readdatadicom, header = nrrd.read(pathdicomnew, index_order='C')

    pathroinew = os.path.join(pathroi, s)
    for g in os.listdir(pathroinew):
        troi = os.path.join(pathroi, s, g)
        readdatanrrd, header2 = nrrd.read(troi, index_order='C')

    patient_features = featureextraction(readdatadicom, readdatanrrd, patient_id=s)
    all_feature_dicts.extend(patient_features)

# Create and save
#dataframe = pd.DataFrame(all_feature_dicts)
#dataframe.to_csv('C:\\Users\\bsbar\\Desktop\\INCRES_ALL_SLICES_NuoviPesi_Ritagliata.csv', index=False)

### RADIMAGENET 2.5D FULL

In [ ]:
#"../PreTrained_Models/RadImageNet-IRV2_notop.h5"
#"../PreTrained_Models/RadImageNet-ResNet50_notop.h5"
#"../PreTrained_Models/RadImageNet-InceptionV3_notop.h5"

# Load the pre trained model, you can choose the network by copy and pasting the paths above
model_path = "../PreTrained_Models/RadImageNet-InceptionV3_notop.h5"
model = load_model(model_path)

# Patients folder
pathdicom = "\\Users\\bsbar\\Desktop\\pazienti_nrrd"
pathroi = "\\Users\\bsbar\\Desktop\\Tesi\\ROI"

def featureextraction(image_array, mask_array, patient_id):
    batch_slices = []
    original_slice_indices = []

    for slice_idx in range(image_array.shape[0]):
        image_slice = image_array[slice_idx]
        mask_slice = mask_array[slice_idx]

        if np.any(mask_slice):
            image_slice = image_slice.astype(np.float32)
            roi_image_resized = x = cv2.resize(image_slice, (224, 224))
            x = image.img_to_array(roi_image_resized)
            min_val = np.min(x)
            max_val = np.max(x)

            # min max norm
            x = (x - min_val) / (max_val - min_val)
            x = np.repeat(x, 3, axis=-1) 

            batch_slices.append(x) 
            original_slice_indices.append(slice_idx)

    batch_slices = np.array(batch_slices)

    base_model_pool_features = model.predict(batch_slices)

    all_features = []
    for i in range(base_model_pool_features.shape[0]):
        feature_map = base_model_pool_features[i]
        feature_map = feature_map.transpose((2, 1, 0))
        
        ## Global Max Pooling
        features = np.max(feature_map, -1)
        features = np.max(features, -1)

        feature_entry = {'Patient': patient_id, 'Slice': original_slice_indices[i]}
        for ind_, f_ in enumerate(features):
            feature_entry[f'Feature_{ind_}'] = f_

        all_features.append(feature_entry)

    return all_features


all_feature_dicts = []

for s in os.listdir(pathdicom):
    print(f"Processing patient: {s}")
    filename = os.path.join(pathdicom, s)

    for t in os.listdir(filename):
        pathdicomnew = os.path.join(pathdicom, s, t)
        readdatadicom, header = nrrd.read(pathdicomnew, index_order='C')

    pathroinew = os.path.join(pathroi, s)
    for g in os.listdir(pathroinew):
        troi = os.path.join(pathroi, s, g)
        readdatanrrd, header2 = nrrd.read(troi, index_order='C')

    patient_features = featureextraction(readdatadicom, readdatanrrd, patient_id=s)
    all_feature_dicts.extend(patient_features)

# Create and save csv
#dataframe = pd.DataFrame(all_feature_dicts)
#dataframe.to_csv('C:\\Users\\bsbar\\Desktop\\INCEPTION_ALL_SLICES_NuoviPesi_FullImage.csv', index=False)
